In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('../input/mobile-price-classification/train.csv')

In [ ]:
data.head(5)

In [ ]:
X=data.iloc[:,0:20]
y=data.iloc[:,-1]

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
#apply selectkbest class to select top ten k best feature
bestfeature= SelectKBest(score_func=chi2,k=10)
fit=(bestfeature.fit(X,y))

In [ ]:
dfscores=pd.DataFrame(fit.scores_)
dfcolumns=pd.DataFrame(X.columns)

In [ ]:
featureScores=pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns=['Specs','Score']

In [ ]:
featureScores

In [ ]:
#1. using SelectKBest selecting top 10 feature
print(featureScores.nlargest(10,'Score'))

In [ ]:
#On basis of feature importancee
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model=ExtraTreesClassifier()
model.fit(X,y)

In [ ]:
print(model.feature_importances_)

In [ ]:
#2. using ExtraTreesClassifier on basis of feature importance
feat_importances=pd.Series(model.feature_importances_,index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

In [ ]:
#Feature selection using correlation
import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
%matplotlib inline

In [ ]:
#Feature selection using correlation
#Filter Method:
#Using Pearson Correlation
plt.figure(figsize=(18,18))
cor = data.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

#corrmat=data.corr()
#top_corr_features= corrmat.index
#plt.figure(figsize=(12,8))
#g=sns.heatmap(data[top_corr_features].corr(),annot=True)

In [ ]:
#3. Correlation with output variable
cor_target = abs(cor["price_range"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.1]
relevant_features

In [ ]:
#4. Wrapper Method:
#i. Backward Elimination
#Adding constant column of ones, mandatory for sm.OLS model
X_1 = sm.add_constant(X)
#Fitting sm.OLS model
model = sm.OLS(y,X_1).fit()
model.pvalues

In [ ]:
#Backward Elimination
cols = list(X.columns)
pmax = 1
while (len(cols)>0):
    p= []
    X_1 = X[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)

In [ ]:
#ii. RFE (Recursive Feature Elimination)
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
model = LinearRegression()
#Initializing RFE model
rfe = RFE(model, 7)
#Transforming data using RFE
X_rfe = rfe.fit_transform(X,y)  
#Fitting the data to model
model.fit(X_rfe,y)
print(rfe.support_)
print(rfe.ranking_)

In [ ]:
#ii. RFE (Recursive Feature Elimination)
cols = list(X.columns)
model = LinearRegression()
#Initializing RFE model
rfe = RFE(model, 10)             
#Transforming data using RFE
X_rfe = rfe.fit_transform(X,y)  
#Fitting the data to model
model.fit(X_rfe,y)              
temp = pd.Series(rfe.support_,index = cols)
selected_features_rfe = temp[temp==True].index
print(selected_features_rfe)

In [ ]:
#no of features
from sklearn.model_selection import train_test_split
nof_list=np.arange(1,21)            
high_score=0
#Variable to store the optimum features
nof=0           
score_list =[]
for n in range(len(nof_list)):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)
    model = LinearRegression()
    rfe = RFE(model,nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)
    model.fit(X_train_rfe,y_train)
    score = model.score(X_test_rfe,y_test)
    score_list.append(score)
    if(score>high_score):
        high_score = score
        nof = nof_list[n]
print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score))

In [ ]:
X=data.iloc[:,0:20]
y=data.iloc[:,-1]

In [ ]:
X.shape,y.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [ ]:
X_train.shape,y_train.shape

In [ ]:
X_test.shape,y_train.shape

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
predict = model.predict(X_test)

In [ ]:
print("coefficient:",model.coef_)
print("intercept:",model.intercept_)
print("coefficient of determination (R^2):%.2f"
      %r2_score(y_test, predict))

In [ ]:
mean_squared_error(y_test, predict)


In [ ]:
plt.scatter(y_test, predict,alpha=0.2)
plt.xlabel('Actual Price')
plt.ylabel('Predicted Price')